In [ ]:
!pip install torch opencv-python pandas ultralytics fastapi uvicorn pyngrok nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00


In [ ]:
# --- Imports ---
import cv2
import torch
import pandas as pd
import datetime
from google.colab.patches import cv2_imshow
from IPython.display import display, HTML, clear_output
import time
from google.colab import files

In [ ]:
# --- Função: processar vídeo e salvar log ---
def process_video(video_path, start_sec, end_sec, output_csv="historico_deteccoes.csv"):
    # Lista para armazenar as detecções de cada frame
    detections_log = []

    # Carrega o modelo YOLOv5 pré-treinado da Ultralytics
    # 'yolov5s' é a versão menor, mais rápida
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    # Configura o modelo para detectar apenas a classe "moto" (ID 3)
    model.classes = [3]

    # Inicializa a captura de vídeo a partir do arquivo
    cap = cv2.VideoCapture(video_path)

    # Verifica se o vídeo foi aberto corretamente
    if not cap.isOpened():
        print(f"Erro ao abrir vídeo: {video_path}")
        return  # Encerra a função se não conseguir abrir o vídeo

    # Define o ponto inicial do vídeo em milissegundos
    cap.set(cv2.CAP_PROP_POS_MSEC, start_sec*1000)
    # Define o tempo final em milissegundos
    end_ms = end_sec*1000

    try:
        # Loop para processar cada frame do vídeo
        while True:
            # Lê um frame
            ret, frame = cap.read()

            # Sai do loop se o vídeo terminou ou se ultrapassou o tempo final
            if not ret or cap.get(cv2.CAP_PROP_POS_MSEC) >= end_ms:
                break

            # Realiza a detecção de objetos no frame
            results = model(frame)
            # Converte os resultados para um DataFrame do pandas
            detections = results.pandas().xyxy[0]

            # Calcula o número de motos detectadas
            num_motos = len(detections)
            # Calcula a confiança média, se houver alguma detecção
            avg_conf = detections['confidence'].mean() if num_motos>0 else 0

            # Adiciona os dados do frame à lista de log
            detections_log.append({
                'timestamp': datetime.datetime.now().isoformat(),
                'num_motos_detectadas': num_motos,
                'confianca_media': avg_conf
            })

            # Loop para desenhar as caixas delimitadoras em cada moto detectada
            for _, det in detections.iterrows():
                # Extrai as coordenadas da caixa delimitadora
                xmin, ymin, xmax, ymax = int(det['xmin']), int(det['ymin']), int(det['xmax']), int(det['ymax'])
                # Extrai a confiança da detecção
                conf = det['confidence']
                # Desenha um retângulo verde ao redor da moto
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                # Adiciona o texto com a confiança acima da caixa
                cv2.putText(frame, f"Moto: {conf:.2f}", (xmin, ymin-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0),2)

            # Exibe o frame no Colab
            cv2_imshow(frame)

    except Exception as e:
        # Captura e imprime qualquer erro ocorrido durante o processamento
        print("Erro:", e)
    finally:
        # Libera o objeto de captura de vídeo para liberar memória
        cap.release()

    # Se houver detecções, salva o log em CSV
    if detections_log:
        pd.DataFrame(detections_log).to_csv(output_csv, index=False)
        print(f"CSV salvo: {output_csv}")
        # Disponibiliza o download do CSV no Colab
        files.download(output_csv)


In [ ]:
# --- Função: mostrar dashboard no Colab ---
def show_dashboard(csv_path, refresh_interval=5, max_updates=None):
    """
    Exibe um dashboard HTML no Colab mostrando os dados do CSV de detecções de motos.

    Parâmetros:
    ----------
    csv_path : str
        Caminho para o arquivo CSV com as detecções.
    refresh_interval : int, opcional
        Intervalo em segundos para atualizar o dashboard (default = 5s).
    max_updates : int ou None, opcional
        Número máximo de atualizações antes de parar. Se None, atualiza indefinidamente.
    """

    # Contador de atualizações realizadas
    count = 0

    # Loop infinito para atualizar o dashboard continuamente
    while True:
        # Limpa a saída anterior do notebook para atualizar a tabela
        clear_output(wait=True)

        # Lê o CSV atualizado
        df = pd.read_csv(csv_path)

        # Exibe o dashboard em HTML
        # Inclui título, intervalo de atualização e a tabela de dados
        display(HTML(f"""
            <h2>Dashboard de Detecções de Motos</h2>
            <p>Atualização a cada {refresh_interval} segundos</p>
            {df.to_html(index=False, border=1, justify='center')}
        """))

        # Aguarda o tempo definido antes de atualizar novamente
        time.sleep(refresh_interval)

        # Incrementa o contador de atualizações
        count += 1

        # Se max_updates estiver definido e o contador atingir o limite, encerra o loop
        if max_updates and count >= max_updates:
            break


In [ ]:

# Caminho do arquivo de vídeo que será processado
video_file = "/video_sprint3.mp4"

# Define o segundo do vídeo onde o processamento deve começar
start_time_sec = 6  # Início do processamento em segundos

# Define o segundo do vídeo onde o processamento deve terminar
end_time_sec = 9    # Fim do processamento em segundos

# Nome do arquivo CSV que armazenará os logs das detecções
# O arquivo ficará salvo em /content/ e pode ser baixado manualmente depois
csv_file = "/content/historico_deteccoes.csv"


In [ ]:
# --- Rodar detecção e salvar CSV ---
# Chama a função process_video para processar o vídeo definido em 'video_file'
# Entre os segundos 'start_time_sec' e 'end_time_sec', detecta motos em cada frame
# Salva os resultados (timestamp, número de motos e confiança média) no arquivo CSV definido em 'csv_file'
process_video(video_file, start_time_sec, end_time_sec, csv_file)

# --- Rodar dashboard interativo ---
# Chama a função show_dashboard para exibir os dados do CSV em um dashboard HTML no Colab
# refresh_interval=5: o dashboard é atualizado a cada 5 segundos
# max_updates=20: o dashboard será atualizado 20 vezes e depois o loop para automaticamente
show_dashboard(csv_file, refresh_interval=5, max_updates=20)
